In [9]:
import os
import os.path

import numpy as np
import pandas as pd

In [2]:
s3 = boto3.resource('s3')
bucket_name = "postie-testing-assets"

In [3]:
test = s3.Bucket(bucket_name)

In [4]:
s3.meta.client.head_bucket(Bucket=bucket_name)

{'ResponseMetadata': {'HTTPHeaders': {'content-type': 'application/xml',
   'date': 'Mon, 16 Oct 2017 18:06:53 GMT',
   'server': 'AmazonS3',
   'transfer-encoding': 'chunked',
   'x-amz-bucket-region': 'us-east-1',
   'x-amz-id-2': 'YhUEo61GDGSwz1qOpFGJl+C9Sxal34XKRYzOI0TF49PsSSGsbGg2Y6xwbf07z+KHIKusPIYkjxE=',
   'x-amz-request-id': 'DDD0C4B61BDF320E'},
  'HTTPStatusCode': 200,
  'HostId': 'YhUEo61GDGSwz1qOpFGJl+C9Sxal34XKRYzOI0TF49PsSSGsbGg2Y6xwbf07z+KHIKusPIYkjxE=',
  'RequestId': 'DDD0C4B61BDF320E',
  'RetryAttempts': 0}}

In [5]:
for key in test.objects.all():
    

2017-07-01.csv
2017-07-02.csv
2017-07-03.csv


In [8]:
keys = s3.list_objects(Bucket=bucket_name)['Contents']
for key in keys:
    s3.download_file(bucket_name, key['Key'], key['Key'])

AttributeError: 's3.ServiceResource' object has no attribute 'list_objects'

## Download the data using

```aws s3 cp --recursive s3://my_bucket_name local_folder```

## Data Issues

1. Column names have spaces, so we need to modify them.
2. Placeholder is blank (NaN) for two files. But is this needed?

## Observations

1. Analyst is correct - July 3rd has sales of $164,065. There are fewer sales on the 3rd than the other two days.

In [40]:
fr = pd.DataFrame()
col_names = ['timestamp', 'website_id', 'customer_id', 'app_version',
       'placeholder', 'checkout_amount', 'url']
data_report = []
for fname in os.listdir("data"):
    ffr = pd.read_csv(os.path.join("data", fname),
                      header=0, names=col_names,
                      infer_datetime_format=True, parse_dates=[0])
    print("date {} has {} sales for rows {}".format(fname, ffr.checkout_amount.sum(), ffr.shape[0]))
    data_report.append({"date": fname.split("."), "sales": ffr.checkout_amount.sum(),
                        "placeholder_nan": sum(ffr.placeholder.isnull()),
                        "n_rows": ffr.shape[0],
                        "n_websites": ffr.website_id.unique().shape[0]})
    fr = fr.append(ffr)

date 2017-07-02.csv has 183294.0 sales for rows 11573
date 2017-07-01.csv has 241491.0 sales for rows 11634
date 2017-07-03.csv has 164065.0 sales for rows 9981


In [41]:
data_report

[{'date': ['2017-07-02', 'csv'],
  'n_rows': 11573,
  'n_websites': 2,
  'placeholder_nan': 11573,
  'sales': 183294.0},
 {'date': ['2017-07-01', 'csv'],
  'n_rows': 11634,
  'n_websites': 2,
  'placeholder_nan': 11634,
  'sales': 241491.0},
 {'date': ['2017-07-03', 'csv'],
  'n_rows': 9981,
  'n_websites': 2,
  'placeholder_nan': 1792,
  'sales': 164065.0}]

In [30]:
fr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33188 entries, 0 to 9980
Data columns (total 7 columns):
timestamp          33188 non-null datetime64[ns]
website_id         33188 non-null int64
customer_id        33188 non-null int64
app_version        33188 non-null float64
placeholder        8189 non-null object
checkout_amount    33188 non-null float64
url                33188 non-null object
dtypes: datetime64[ns](1), float64(2), int64(2), object(2)
memory usage: 2.0+ MB
